## Check List 4 - MotionFilter
* Motion filters are quick decision makers that prevents trying to plan a infeasible motion.
* Here, GraspChecker is tested with some example cases  
* **4-A.1 GraspChecker**  
  - pick, place, sweep
    - single process
    - multi process

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=False)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepTool

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name="indy0_tcp", 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name="indy0_tcp",
                              dims=(0.02,0.02,0.1), center=(0,0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name="indy0_tcp", 
                              dims=(0.02,0.02,0.1), center=(0,-0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", 
                     _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepTool, point=(0,0,0), rpy=(0,0,0))

##### create_subject

In [7]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepPoint
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

##### update_state
* update_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [8]:
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('floor', 0)


##### prepare motion planner

In [9]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)

##### Create GraspChecker

In [10]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.constraint.constraint_common import calc_redundancy
mfilter = GraspChecker(pscene)

##### pick scenarios

In [11]:
obj_name, handle_name, actor_name, actor_geo = "box1", "handle1", "grip0", "gripper"
actor = pscene.actor_dict[actor_name]
obj = pscene.subject_dict[obj_name]
handle = obj.action_points_dict[handle_name]
binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
constrained = False

In [12]:
scenario = "vertial-access"
from_state=initial_state
Q_dict = list2dict(from_state.Q, gscene.joint_names)
to_state = from_state.copy(pscene)
to_state.set_binding_state(
    ((obj_name, handle_name, actor_name, actor_geo), 
     to_state.binding_state[1]), 
    pscene)
redundancy_dict = {obj_name:{handle_name:{"w":np.pi}, 
                             actor_name:{"w":0}}}
redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                     (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                    }

pscene.set_object_state(from_state)
gscene.show_pose(from_state.Q)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

Traj, LastQ, error, success = mplan.plan_algorithm(
    from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
    pick_state = pscene.rebind_all(binding_list, LastQ)
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
Result match for vertial-access: True / True


In [13]:
scenario = "vertical-access-with-obstacle"

obstacle.set_offset_tf(center=(0.3,0.4,0.2)) # move obstacle
gscene.update_markers_all()

pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
Traj, LastQ, error, success = mplan.plan_algorithm(
    from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
gscene.show_motion(Traj, period=0.05)
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"
obstacle.set_offset_tf(center=(0.5, 0.4, 0.031)) # return obstacle
gscene.update_markers_all()

try transition motion
transition motion tried: False
Result match for vertical-access-with-obstacle: False / False


In [14]:
scenario = "horizontal-access"

redundancy_dict = {obj_name:{handle_name:{"w":np.pi/2}, 
                             actor_name:{"w":0}}}
redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                     (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                    }

pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
Traj, LastQ, error, success = mplan.plan_algorithm(
    from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
gscene.show_motion(Traj, period=0.05)
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: False
Result match for horizontal-access: False / False


##### place scenarios

In [15]:
obj_name, handle_name, actor_name, actor_geo = "box1", "bottom", "goal", "goal"
actor = pscene.actor_dict[actor_name]
obj = pscene.subject_dict[obj_name]
handle = obj.action_points_dict[handle_name]
binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
constrained = False

In [16]:
scenario = "vertial-access"
from_state=pick_state
Q_dict = list2dict(from_state.Q, gscene.joint_names)
to_state = from_state.copy(pscene)
to_state.set_binding_state(
    ((obj_name, handle_name, actor_name, actor_geo), 
     to_state.binding_state[1]), 
    pscene)
redundancy_dict = {obj_name:{handle_name:{"w":0}, 
                             actor_name:{"w":0}}}
redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                     (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                    }

pscene.set_object_state(from_state)
gscene.show_pose(crob.home_pose)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

Traj, LastQ, error, success = mplan.plan_algorithm(
    from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
    place_state = pscene.rebind_all(binding_list, LastQ)
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
Result match for vertial-access: True / True


In [17]:
scenario = "vertical-access-with-obstacle"
obstacle.set_offset_tf(center=(0.3,-0.4,0.031)) # move obstacle
gscene.update_markers_all()

pscene.set_object_state(from_state)
gscene.show_pose(crob.home_pose)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
Traj, LastQ, error, success = mplan.plan_algorithm(
    from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
gscene.show_motion(Traj)

obstacle.set_offset_tf(center=(0.5, 0.4, 0.031)) # return obstacle
gscene.update_markers_all()
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: False
Result match for vertical-access-with-obstacle: False / False


##### sweep scenario 1: reaching first sweep point

In [18]:
obj_name, handle_name, actor_name, actor_geo = "sweep", "wp1", "sweep_face", "sweep_face"
actor = pscene.actor_dict[actor_name]
obj = pscene.subject_dict[obj_name]
handle = obj.action_points_dict[handle_name]
binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
constrained = False

In [19]:
scenario = "reach wp1"
from_state=place_state.copy(pscene)
from_state.Q = crob.home_pose
Q_dict = list2dict(from_state.Q, gscene.joint_names)
to_state = from_state.copy(pscene)
to_state.set_binding_state(
    (to_state.binding_state[1], 
     (obj_name, handle_name, actor_name, actor_geo)), 
    pscene)
redundancy_dict = {obj_name:{handle_name:{"w":0}, 
                             actor_name:{"w":0}}}
redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                     (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                    }

pscene.set_object_state(from_state)
gscene.show_pose(from_state.Q)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
for _ in range(100):
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
    if success and np.abs(LastQ[0])<np.pi/2: # else, it will fail in sweep motion
        break
if success:
    if np.abs(LastQ[0])<np.pi/2:
        print("OK to go")
        pscene.set_object_state(from_state)
        gscene.show_motion(Traj, period=0.05)
        sweep1_state = pscene.rebind_all(binding_list, LastQ)
    else:
        print("Sweep will fail. Please try again from this step.")
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
OK to go
Result match for reach wp1: True / True


In [20]:
scenario = "reach wp1 with obstacle"
from_state=place_state.copy(pscene)
obstacle.set_offset_tf(center=(0.4,-0.3,0.2)) # move obstacle
gscene.update_markers_all()

pscene.set_object_state(place_state)
gscene.show_pose(from_state.Q)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
Traj, LastQ, error, success = mplan.plan_algorithm(
    from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
gscene.show_motion(Traj, period=0.05)
    
obstacle.set_offset_tf(center=(0.5, 0.4, 0.031)) # return obstacle
gscene.update_markers_all()
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
Result match for reach wp1 with obstacle: True / True


##### sweep scenario 2: sweep to second sweep point

In [21]:
obj_name, handle_name, actor_name, actor_geo = "sweep", "wp2", "sweep_face", "sweep_face"
actor = pscene.actor_dict[actor_name]
obj = pscene.subject_dict[obj_name]
handle = obj.action_points_dict[handle_name]
binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
constrained = True     # by setting constrained = True, GraspChecker will check collision in interpolated poses too.

In [22]:
scenario = "sweep to wp2"
from_state=sweep1_state.copy(pscene)
Q_dict = list2dict(from_state.Q, gscene.joint_names)
to_state = from_state.copy(pscene)
to_state.set_binding_state(
    (to_state.binding_state[1], 
     (obj_name, handle_name, actor_name, actor_geo)), 
    pscene)
redundancy_dict = {obj_name:{handle_name:{"w":0}, 
                             actor_name:{"w":0}}}
redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                     (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                    }

pscene.set_object_state(from_state)
gscene.show_pose(from_state.Q)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
Traj, LastQ, error, success = mplan.plan_algorithm(
    from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values,
    timeout=30)
if success:
    gscene.show_motion(Traj, period=0.05)
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try constrained motion
constrained motion tried: True
Result match for sweep to wp2: True / True


In [23]:
scenario = "sweep to wp2 with obstacle"
obstacle.set_offset_tf(center=(0.5, 0.0, 0.2)) # return obstacle
gscene.update_markers_all()

pscene.set_object_state(from_state)
gscene.show_pose(from_state.Q)
success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
Traj, LastQ, error, success = mplan.plan_algorithm(
    from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values,
    timeout=30)

gscene.show_motion(Traj, period=0.05)
obstacle.set_offset_tf(center=(0.5, 0.4, 0.031)) # return obstacle
gscene.update_markers_all()
print("Result {} for {}: {} / {}".format("match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try constrained motion
constrained motion tried: False
Result match for sweep to wp2 with obstacle: False / False


##### planning: pick & place & sweep

In [24]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene, 
                                        motion_filters=[GraspChecker(pscene)]))
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene))

##### - single process

In [25]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (0.02/100.0 s, steps/err: 14(20.3280448914 ms)/0.00217988449718)
try: 1 - ('floor', 1)->('floor', 2)
try constrained motion
constrained motion tried: True
result: 1 - ('floor', 1)->('floor', 2) = success
branching: 1->2 (0.05/100.0 s, steps/err: 28(28.1670093536 ms)/0.00190506532324)
try: 2 - ('floor', 2)->('gripper', 2)
result: 2 - ('floor', 2)->('gripper', 2) = fail
try: 1 - ('floor', 1)->('floor', 2)
try constrained motion
constrained motion tried: False
result: 1 - ('floor', 1)->('floor', 2) = fail
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->3 (10.08/100.0 s, steps/err: 13(18.0420875549 ms)/0.00131604118665)
try: 1 - ('floor', 1)->('floor', 2)
try constrained motion
constrained motion tried: True
result: 1 - ('floor', 1)->('fl

##### - multi process

In [26]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

Use 20/20 agents
try: 0 - ('floor', 0)->('gripper', 0)
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (0.09/100.0 s, steps/err: 7(32.3140621185 ms)/0.00138280408796)
try: 1 - ('floor', 1)->('floor', 2)
try constrained motion
try: 1 - ('floor', 1)->('floor', 2)
try constrained motion
constrained motion tried: True
result: 1 - ('floor', 1)->('floor', 2) = success
branching: 1->2 (0.15/100.0 s, steps/err: 45(46.0939407349 ms)/0.000789607818582)
try: 2 - ('floor', 2)->('gripper', 2)
try transition motion
constrained motion tried: True
result: 1 - ('floor', 1)->('floor', 2) = success
branching: 1->3 (0.17/100.0 s, steps/err: 72(31.7461490631 ms)/0.00186186985354)
try: 3 - ('floor', 2)->('gripper', 2)
try transition motion
transition motion tried: True
transition motion tried: True
=============== try reaching goal from ('gripper', 2) =====

result: 0 - ('floor', 0)->('floor', 1) = success
joint motion tried: True
======================= terminated 8: first answer acquired from other agent ===============================
======================= terminated 2: first answer acquired ===============================
++ adding return motion to acquired answer ++
constrained motion tried: True
branching: 0->20 (0.66/100.0 s, steps/err: 8(102.61797905 ms)/0.0018770323337)
======================= terminated 15: first answer acquired from other agent ===============================
joint motion tried: True
result: 15 - ('floor', 1)->('floor', 2) = success
try joint motion
======================= terminated 1: first answer acquired ===============================
result: 0 - ('floor', 0)->('gripper', 0) = fail
++ adding return motion to acquired answer ++
joint motion tried: True
branching: 15->23 (0.7/100.0 s, steps/err: 44(73.734998703 ms)/0.00191569115022)
======================= terminated 11: first answer acquired from other age